In [1]:
%env WANDB_PROJECT=aspect_explain_root_cause

env: WANDB_PROJECT=aspect_explain_root_cause


In [2]:
from project_dataset import load_dataset


In [3]:
from dataclasses import dataclass

@dataclass
class Args:
    model_name = "Salesforce/codet5p-220m"
    num_proc = 8
    batch_size = 5
    max_src_length = 1200
    max_des_length = 153
    data_cols = ["CVE ID", "explain", "func_before", "processed_func"]
    save_dir = 'tf_board'
    epochs = 100
    grad_acc_steps = 4
    lr = 5e-5
    log_freq = 10
    local_rank = -1
    deepspeed = None
    fp16 = False
    lr_warmup_steps = 200
    weight_decay = 0.05
    task = "root_cause"
    
args = Args()

In [4]:
ds = load_dataset(args.task)

In [5]:
import os

os.makedirs(args.save_dir, exist_ok=True)

In [6]:
from transformers import AutoTokenizer
codet5p_tokenizer = AutoTokenizer.from_pretrained(args.model_name)

In [7]:
import numpy as np

def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["func_before"]]
    target = [' '.join(ex) for ex in examples["explain"]]

    input_feature = codet5p_tokenizer(source, max_length=args.max_src_length, padding="max_length", truncation=True)
    labels = codet5p_tokenizer(target, max_length=args.max_des_length, padding="max_length", truncation=True)

    lables = labels["input_ids"].copy()
    # lables = np.where(lables != codet5p_tokenizer.pad_token_id, lables, -100)

    return {  "input_ids": input_feature["input_ids"],
              "attention_mask": input_feature["attention_mask"],
              "labels": lables}


tokenized_ds = ds.map(
  preprocess_function,
  remove_columns=args.data_cols,
  batched=True,
  num_proc=args.num_proc,
  batch_size=args.batch_size)

tokenized_ds

Map (num_proc=8):   0%|          | 0/3431 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/382 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/954 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3431
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 382
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 954
    })
})

In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name).to(device)

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  codet5p_tokenizer,
  model=model,
  return_tensors="pt")

In [10]:
import evaluate
import numpy as np

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = codet5p_tokenizer(arg)
  return codet5p_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, codet5p_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = codet5p_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = codet5p_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  text_preds = ["\n".join(p for p in text_preds)]
  text_labels = ["\n".join(l for l in text_labels)]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

Using the latest cached version of the module from /data/quy/cache/modules/evaluate_modules/metrics/evaluate-metric--rouge/b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Sun Jul  9 23:52:56 2023) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


In [11]:
# for testing, show first 5 rows

from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()
  break

metrics_func([preds, labels])

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Token indices sequence length is longer than the specified maximum sequence length for this model (760 > 512). Running this sequence through the model will result in indexing errors


{'rouge1': 0.45290581162324645,
 'rouge2': 0.19879518072289157,
 'rougeL': 0.22244488977955912,
 'rougeLsum': 0.46436781609195404}

In [12]:
preds.shape, labels.shape

(torch.Size([5, 153]), torch.Size([5, 153]))

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        report_to='wandb',
        output_dir=args.save_dir,
        overwrite_output_dir=False,

        do_train=True,
        save_strategy='epoch',
        do_eval=True,

        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=args.grad_acc_steps,

        learning_rate=args.lr,
        weight_decay=args.weight_decay,
        warmup_steps=args.lr_warmup_steps,

        logging_dir=args.save_dir,
        logging_first_step=True,
        logging_steps=args.log_freq,
        save_total_limit=1,

        dataloader_drop_last=True,
        dataloader_num_workers=args.num_proc,

        local_rank=args.local_rank,
        deepspeed=args.deepspeed,
        fp16=args.fp16,
    )

In [14]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"],
    tokenizer = codet5p_tokenizer,
)

trainer.train()

/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dongchirua. Use `wandb login --relogin` to force relogin


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,9.255700
10,8.912100
20,7.640500
30,6.316300
40,5.926000
50,5.581500
60,5.230900
70,4.774800
80,4.519800
90,4.366300


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-package

TrainOutput(global_step=8500, training_loss=0.15234780522674213, metrics={'train_runtime': 25420.0777, 'train_samples_per_second': 13.497, 'train_steps_per_second': 0.334, 'total_flos': 4.8526331904e+17, 'train_loss': 0.15234780522674213, 'epoch': 99.13})

In [15]:
trainer.save_model(f'{args.save_dir}/{args.task}')

## measure on test set

In [14]:
model.load_state_dict(torch.load(f'{args.save_dir}/{args.task}/pytorch_model.bin'))

<All keys matched successfully>

In [16]:
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=50,
  num_workers=args.num_proc)

gens = []
references = []

for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
          batch["input_ids"].to(device),
          max_length=args.max_des_length,
        ).cpu()
    labels = batch["labels"].cpu()
    gens += preds
    references += labels

In [17]:
gens = codet5p_tokenizer.batch_decode(gens, skip_special_tokens=True)
references = codet5p_tokenizer.batch_decode(references, skip_special_tokens=True)

In [18]:
from commons import clean_generated_str

In [19]:
gens = [clean_generated_str(i) for i in gens]
references = [clean_generated_str(i) for i in references]

In [20]:
results = rouge_metric.compute(predictions=gens, references=references)
results

{'rouge1': 0.5907898368300832,
 'rouge2': 0.5614364938463559,
 'rougeL': 0.5892991453001686,
 'rougeLsum': 0.5902769937521375}